In [ ]:
#!pip install tweepy
#!pip install configparser
#!pip install wordcloud
#!pip install nltk
#!pip install textblob
#!pip install estnltk==1.7.1
#!pip install gensim 

In [1]:
import configparser
import tweepy
import pandas as pd
import re
import csv
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import datetime

## Authentication

In [ ]:
import configparser
config = configparser.RawConfigParser()
config.read('tweet.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
bearer_token =  config['twitter']['bearer_token']

## Tweepy setup

- API guide: https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9  
- Tweepy Documentation: https://docs.tweepy.org/en/stable/index.html   
- Twitter API v2 data dictionary: https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet

In [ ]:
import tweepy
client = tweepy.Client(bearer_token, wait_on_rate_limit = True)

auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth, wait_on_rate_limit= True, retry_count=12, retry_delay=5, retry_errors=set([503, 500]))

## Collect usernames based on 'leaders' names

In [2]:
users = pd.read_excel('./leaders_names/final.xlsx', 'clean')

In [3]:
print(len(users))
users.head() # with duplicates

1865


,nimi,erakond,from,timeframe,media,clean_time
0,Toomas Hendrik Ilves,NaN,presidents,2006-2016,NaN,2016
1,Kersti Kaljulaid,NaN,presidents,2016-2021,NaN,2021
2,Alar Karis,NaN,presidents,2021_ongoing,NaN,2023
3,Arto Aas,REF,parliament_13_comp,24.03.2015-29.03.2019,NaN,2019
4,Yoko Alender,REF,parliament_13_comp,24.03.2015-29.03.2019,NaN,2019


In [4]:
latest = {}

for i, row in users.iterrows():
    name = row['nimi']
    
    if name in latest:
        last_value = latest[name]['clean_time']
        curr_value = row['clean_time']
        last_media = latest[name]['media']
        curr_media = row['media']
        
        if curr_value > last_value:
            latest[name] = row
        
        if last_media == 'EAL' and curr_media != 'EAL':
            latest[name] = row
            
    else:
        latest[name] = row

In [5]:
cleaned_users = pd.DataFrame.from_dict(latest, orient='index')
cleaned_users = cleaned_users.reset_index()

In [ ]:
for users in tweepy.Cursor(api.search_users, q='politician name').items(1):
    print(users.status.created_at)

2016-05-04 12:02:58+00:00


In [6]:
print('Total rows',len(cleaned_users))
cleaned_users.head()

Total rows 1671


,index,nimi,erakond,from,timeframe,media,clean_time
0,Toomas Hendrik Ilves,Toomas Hendrik Ilves,NaN,presidents,2006-2016,NaN,2016
1,Kersti Kaljulaid,Kersti Kaljulaid,NaN,presidents,2016-2021,NaN,2021
2,Alar Karis,Alar Karis,NaN,presidents,2021_ongoing,NaN,2023
3,Arto Aas,Arto Aas,REF,parliament_13_comp,24.03.2015-29.03.2019,NaN,2019
4,Yoko Alender,Yoko Alender,REF,parliament_14_comp,20.03.2019_ongoing,NaN,2023


In [ ]:
twitter_names = []
follower_count = []
tweet_count = []
last_tweet_times = []

# start counting time
start_time = time.time()
print("Total rows", len(cleaned_users))
for i,row in cleaned_users.iterrows():
    current_name = ''
    current_follower_count = None
    current_tweet_count = None
    current_last_tweet_time = None
    
    if (i % 20 == 0):
        print("Currently done rows count", str(i))

    # This .items(1) takes the first match
    for users in tweepy.Cursor(api.search_users, q=row['nimi']).items(1):
        current_name = users.screen_name
        current_follower_count = users.followers_count
        current_tweet_count = users.statuses_count
        if hasattr(users, 'status'):
            current_last_tweet_time = users.status.created_at
    
    twitter_names.append(current_name)
    follower_count.append(current_follower_count)
    tweet_count.append(current_tweet_count)
    last_tweet_times.append(current_last_tweet_time)

# end counting time
end_time = time.time()                   
print("--- %s seconds ---" % (end_time - start_time))

Total rows 1671
Currently done rows count 0
Currently done rows count 20
Currently done rows count 40
Currently done rows count 60
Currently done rows count 80
Currently done rows count 100
Currently done rows count 120
Currently done rows count 140
Currently done rows count 160
Currently done rows count 180
Currently done rows count 200
Currently done rows count 220
Currently done rows count 240
Currently done rows count 260
Currently done rows count 280
Currently done rows count 300
Currently done rows count 320
Currently done rows count 340
Currently done rows count 360
Currently done rows count 380
Currently done rows count 400
Currently done rows count 420
Currently done rows count 440
Currently done rows count 460
Currently done rows count 480
Currently done rows count 500
Currently done rows count 520
Currently done rows count 540
Currently done rows count 560
Currently done rows count 580
Currently done rows count 600
Currently done rows count 620
Currently done rows count 640


Rate limit reached. Sleeping for: 619


Currently done rows count 1060
Currently done rows count 1080
Currently done rows count 1100
Currently done rows count 1120
Currently done rows count 1140
Currently done rows count 1160
Currently done rows count 1180
Currently done rows count 1200
Currently done rows count 1220
Currently done rows count 1240
Currently done rows count 1260
Currently done rows count 1280
Currently done rows count 1300
Currently done rows count 1320
Currently done rows count 1340
Currently done rows count 1360
Currently done rows count 1380
Currently done rows count 1400
Currently done rows count 1420
Currently done rows count 1440
Currently done rows count 1460
Currently done rows count 1480
Currently done rows count 1500
Currently done rows count 1520
Currently done rows count 1540
Currently done rows count 1560
Currently done rows count 1580
Currently done rows count 1600
Currently done rows count 1620
Currently done rows count 1640
Currently done rows count 1660
--- 1145.6946663856506 seconds ---


In [ ]:
df_users = cleaned_users
df_users['twitter_usernames'] = twitter_names
df_users['twitter_followers'] = follower_count
df_users['twitter_tweet_count'] = tweet_count
df_users['last_tweet_time'] = last_tweet_times

In [ ]:
count = 0
for name in twitter_names:
    if len(name) != 0:
        count += 1
        
print('From', str(len(cleaned_users)), 'we are left with', str(count), 'users')

From 1671 we are left with 878 users


In [ ]:
df_users.head()

,index,nimi,erakond,from,timeframe,media,clean_time,twitter_usernames,twitter_followers,twitter_tweet_count,last_tweet_time
0,Toomas Hendrik Ilves,Toomas Hendrik Ilves,NaN,presidents,2006-2016,NaN,2016,IlvesToomas,190468.0,181887.0,2023-01-07 09:30:10+00:00
1,Kersti Kaljulaid,Kersti Kaljulaid,NaN,presidents,2016-2021,NaN,2021,KerstiKaljulaid,62168.0,1781.0,2022-12-15 13:34:11+00:00
2,Alar Karis,Alar Karis,NaN,presidents,2021_ongoing,NaN,2023,AlarKaris,49265.0,552.0,2023-01-01 14:25:24+00:00
3,Arto Aas,Arto Aas,REF,parliament_13_comp,24.03.2015-29.03.2019,NaN,2019,ArtoAas,2977.0,5622.0,2023-01-05 08:07:05+00:00
4,Yoko Alender,Yoko Alender,REF,parliament_14_comp,20.03.2019_ongoing,NaN,2023,YokoAlender,1636.0,3345.0,2022-12-13 13:17:38+00:00


In [ ]:
df_users.to_csv('twitter_users_new.csv', encoding="utf-8")

Manually deleted the rows where name and twitter account didn't match.

In [208]:
cleaned_twitter_users = pd.read_excel('twitter_users_clean.xlsx', 'Sheet1')

## Merge new users with cleaned
Since I manually removed fake usernames in cleaned twitter users, but have regenereated the twitter_users with some new columns, we will merge to the cleaned users the new data, to avoid having to manually clean fake usernames again

In [209]:
df_users = pd.read_csv('twitter_users_new.csv')
cleaned_twitter_users.head()

,index,nimi,erakond,from,timeframe,media,clean_time,twitter_usernames,twitter_followers,twitter_tweet_count
0,Urmas Reinsalu,Urmas Reinsalu,I,Gov_52_comp,18.07.2022_ongoing,NaN,2023,UrmasReinsalu,11332.0,1057.0
1,Hendrik Johannes Terras,Hendrik Johannes Terras,E200,additional,NaN,NaN,2023,terrashendrikj,72.0,1.0
2,Toomas Hendrik Ilves,Toomas Hendrik Ilves,NaN,presidents,2006-2016,NaN,2016,IlvesToomas,190451.0,181851.0
3,Kersti Kaljulaid,Kersti Kaljulaid,NaN,presidents,2016-2021,NaN,2021,KerstiKaljulaid,62189.0,1781.0
4,Alar Karis,Alar Karis,NaN,presidents,2021_ongoing,NaN,2023,AlarKaris,49262.0,552.0


In [210]:
df_users.head()

,Unnamed: 0,index,nimi,erakond,from,timeframe,media,clean_time,twitter_usernames,twitter_followers,twitter_tweet_count,last_tweet_time
0,0,Toomas Hendrik Ilves,Toomas Hendrik Ilves,NaN,presidents,2006-2016,NaN,2016,IlvesToomas,190468.0,181887.0,2023-01-07 09:30:10+00:00
1,1,Kersti Kaljulaid,Kersti Kaljulaid,NaN,presidents,2016-2021,NaN,2021,KerstiKaljulaid,62168.0,1781.0,2022-12-15 13:34:11+00:00
2,2,Alar Karis,Alar Karis,NaN,presidents,2021_ongoing,NaN,2023,AlarKaris,49265.0,552.0,2023-01-01 14:25:24+00:00
3,3,Arto Aas,Arto Aas,REF,parliament_13_comp,24.03.2015-29.03.2019,NaN,2019,ArtoAas,2977.0,5622.0,2023-01-05 08:07:05+00:00
4,4,Yoko Alender,Yoko Alender,REF,parliament_14_comp,20.03.2019_ongoing,NaN,2023,YokoAlender,1636.0,3345.0,2022-12-13 13:17:38+00:00


In [211]:
print(len(cleaned_twitter_users))
print(len(df_users))

1540
1671


In [212]:
subset_df = df_users[['nimi', 'last_tweet_time']]
subset_df.head()

,nimi,last_tweet_time
0,Toomas Hendrik Ilves,2023-01-07 09:30:10+00:00
1,Kersti Kaljulaid,2022-12-15 13:34:11+00:00
2,Alar Karis,2023-01-01 14:25:24+00:00
3,Arto Aas,2023-01-05 08:07:05+00:00
4,Yoko Alender,2022-12-13 13:17:38+00:00


In [213]:
clean_merged_users = pd.merge(cleaned_twitter_users, subset_df, on=['nimi','nimi'])
clean_merged_users.head()

,index,nimi,erakond,from,timeframe,media,clean_time,twitter_usernames,twitter_followers,twitter_tweet_count,last_tweet_time
0,Urmas Reinsalu,Urmas Reinsalu,I,Gov_52_comp,18.07.2022_ongoing,NaN,2023,UrmasReinsalu,11332.0,1057.0,2023-01-06 13:15:19+00:00
1,Hendrik Johannes Terras,Hendrik Johannes Terras,E200,additional,NaN,NaN,2023,terrashendrikj,72.0,1.0,2022-08-26 09:03:06+00:00
2,Toomas Hendrik Ilves,Toomas Hendrik Ilves,NaN,presidents,2006-2016,NaN,2016,IlvesToomas,190451.0,181851.0,2023-01-07 09:30:10+00:00
3,Kersti Kaljulaid,Kersti Kaljulaid,NaN,presidents,2016-2021,NaN,2021,KerstiKaljulaid,62189.0,1781.0,2022-12-15 13:34:11+00:00
4,Alar Karis,Alar Karis,NaN,presidents,2021_ongoing,NaN,2023,AlarKaris,49262.0,552.0,2023-01-01 14:25:24+00:00


### Correct year column in data

In [214]:
years = []

for i, row in clean_merged_users.iterrows():
    if pd.isna(row['last_tweet_time']):
        years.append(row['last_tweet_time'])
        continue
    element = datetime.datetime.strptime(row['last_tweet_time'],"%Y-%m-%d %H:%M:%S%z")
    years.append(element.year)

In [215]:
clean_merged_users['last_tweet_year'] = years
clean_merged_users.head()

,index,nimi,erakond,from,timeframe,media,clean_time,twitter_usernames,twitter_followers,twitter_tweet_count,last_tweet_time,last_tweet_year
0,Urmas Reinsalu,Urmas Reinsalu,I,Gov_52_comp,18.07.2022_ongoing,NaN,2023,UrmasReinsalu,11332.0,1057.0,2023-01-06 13:15:19+00:00,2023.0
1,Hendrik Johannes Terras,Hendrik Johannes Terras,E200,additional,NaN,NaN,2023,terrashendrikj,72.0,1.0,2022-08-26 09:03:06+00:00,2022.0
2,Toomas Hendrik Ilves,Toomas Hendrik Ilves,NaN,presidents,2006-2016,NaN,2016,IlvesToomas,190451.0,181851.0,2023-01-07 09:30:10+00:00,2023.0
3,Kersti Kaljulaid,Kersti Kaljulaid,NaN,presidents,2016-2021,NaN,2021,KerstiKaljulaid,62189.0,1781.0,2022-12-15 13:34:11+00:00,2022.0
4,Alar Karis,Alar Karis,NaN,presidents,2021_ongoing,NaN,2023,AlarKaris,49262.0,552.0,2023-01-01 14:25:24+00:00,2023.0


### Create list of valid usernames

In [216]:
valid_usernames = []

for idx, user in clean_merged_users.iterrows():
    username = user['twitter_usernames']
    tweet_count = user['twitter_tweet_count']
    last_tweet_year = user['last_tweet_year']


    user_has_no_username = username == '' or pd.isna(username) # check if twitter username exists for a certain person
    user_has_no_tweets = tweet_count == 0 # check if user has tweets
    user_has_not_tweeted_lately = pd.isna(last_tweet_year) or int(last_tweet_year) < 2022 # check if last tweet made in 2022
    if user_has_no_username or user_has_no_tweets or user_has_not_tweeted_lately:
        continue
        
    valid_usernames.append(username)

In [217]:
len(valid_usernames)

232

### Remove those usernames whose data we already have
I ran the search for about a day, but it was taking too long. To speed it up I added more filtering to avoid doing
unneccesary queries, that are not needed. Every query increase the chance of getting 15 minute cooldown. The filtering
removed those users that do not have any tweets, or that have not tweeted since 2021.
Since I have some data collected in 24hrs I do not want to wait for it to be fetched again, so here we are removing those users 
from who we already have data.

In [218]:
print(len(valid_usernames))

232


In [219]:
data_found = pd.read_csv('until_politicianx.csv')
usernames_from_file = list(set(data_found['user']))
print(len(usernames_from_file))

19


In [220]:
filtered_usernames = [username for username in valid_usernames if username not in usernames_from_file]
print("Usernames left to query", str(len(filtered_usernames)))

Usernames left to query 213
